<a href="https://www.inove.com.ar"><img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/PA%20Banner.png" width="1000" align="center"></a>


# Ejercicio de clasificación con redes neuronales profundas (DNN)

Ejemplo de clasificación utilizando redes neuronales para la clasificación de imagenes<br>

v1.1

### **Objetivos**
*   Estudiar el dataset de mnist que contiene 70.000 imagenes de números escritos a mano.
* Visualizar las imágenes a analizar.
* Normalizar la imágenes.
* Comparar tres redes neuronales.
* Transformar la salida a categorical.
* Construir, entrenar y evaluar al modelo con una Red Neuronal.
* Validar el modelo con una imagen externa.

In [ ]:
#Librerias a implementar
import os
import platform

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import keras
from keras.models import Sequential
#from keras.utils import to_categorical  
from keras.utils.np_utils import to_categorical # Si esto no funciona, probar con el import anterior

import matplotlib.image as mpimg 

# Recolectar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline1.png" width="1000" align="middle">

### `MNIST dataset`
Contiene 70.000 imagenes de números escritos a mano (números del 0 al 9, 10 dígitos). Cada imagen es de 28x28 píxeles en escala de grises (1 canal o 1 nivel de profundidad). Es uno de los dataset más utilizados para poner a prueba algoritmos de clasificación de imagenes.<br> [Dataset source](https://keras.io/api/datasets/mnist/)
- La entrada (X) es una variable imagen de 28x28
- La salida (y) es el dígito que representa la imagen en cuestión, un número de 0 al 9


## Código de carga del dataset mnist

In [ ]:
# Importar mnist de keras.datasets
from keras.datasets import mnist

# Leer el dataset de mnist.
# Viene con los datos separados para entrenar y evaluar.
# De objeto importado  "mnist" utilizar el método load_data()
(data_X_train, data_y_train), (data_X_test, data_y_test) = mnist.load_data()

In [ ]:
# plt alias de Matplotlib.
# Método figure() crea el espacio para dibujar.
# Con figsize=(16,9) se define el ancho y alto del dibujo
fig = plt.figure(figsize=(16,9))

# Bucle que itera 50 veces para mostrar las primeras 50 imágenes del dataset
for i in range(50):
    
    # ax gráfico que mostrará las imágenes en 5 filas y 10 columnas
    # En cada iteración va ubicando la imagen en la siguiente posición (i+1)
    ax = fig.add_subplot(5, 10, i+1)
    
    # .axis('off') elimina el recuadro de cada imagen
    ax.axis('off')

    # Muestra las 50 imágenes de la variable data_X_train en el espacio del dibujo
    plt.imshow(data_X_train[i], cmap='Greys')

# Muestra la figura
plt.show()

In [ ]:
# plt, alias de Matplotlib 
# Muestra la primer imagen (data_X_train[0] )de la variable data_X_train en el espacio del dibujo.
# cmap='gray', escala de grises
plt.imshow(data_X_train[0], cmap='gray')

# Agrega título concatenando el número a mostrar de la primer posición
plt.title("Número: " + str(data_y_train[0]))

# Muestra la imagen
plt.show()

# Procesar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline2.png" width="1000" align="middle">

In [ ]:
# Observar como está representada la imagen, ver fila del medio (14)
# data_X_train[0] información de la primer imagen
# [14, :] muestra la fila 14 y todas las columnas (:)
print(data_X_train[0][14, :])

In [ ]:
# Por los resultados podemos ver que la imagen está representada de 0 a 255
# Normalizamos los datos para que se encuentren entre 0 y 1
X_train_norm = data_X_train / 255
X_test_norm = data_X_test / 255

In [ ]:
# shape[0], devuelve cantidad de datos en observacion de la primer imagen.
print('Cantidad de datos en observacion:', X_train_norm.shape[0])

In [ ]:
# Muetra las dimensiones de la primer imagen normalizada.
print('Tamaño de la imagen:', X_train_norm[0].shape)

# Explorar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline3.png" width="1000" align="middle">

In [ ]:
# Observar el los primeros 10 datos del dataset de y_train
print(data_y_train[:10])

## Exploramos los primeros diez 8 del dataset para train

In [ ]:
# plt alias de Matplotlib.
# Método figure() crea el espacio para dibujar.
# Con figsize=(16,9) se define el ancho y alto del dibujo
fig = plt.figure(figsize=(16,9))

#contador
j = 0

# Itera 10 veces
for i in range(10):

    # ax, gráfico que mostrará las imágenes en 1 filas y 10 columnas
    # En cada iteración va ubicando la imagen en la siguiente posición (i+1)
    ax = fig.add_subplot(1, 10, i+1)

    # .axis('off') elimina el recuadro de cada imagen
    ax.axis('off')

    # Bucle que verifica que la imagen sea igual a 8
    while True:
        if data_y_train[j] == 8:

            # Muestra la imagen con escala de grises
            ax.imshow(X_train_norm[j], cmap='Greys')
            j += 1
            break
        j += 1
plt.show()

## Exploramos los primeros diez 8 del dataset para test

In [ ]:
# plt alias de Matplotlib.
# Método figure() crea el espacio para dibujar.
# Con figsize=(16,9) se define el ancho y alto del dibujo
fig = plt.figure(figsize=(16,9))

#contador
j = 0

# Itera 10 veces
for i in range(10):

    # ax, gráfico que mostrará las imágenes en 1 filas y 10 columnas
    # En cada iteración va ubicando la imagen en la siguiente posición (i+1)
    ax = fig.add_subplot(1, 10, i+1)

    # .axis('off') elimina el recuadro de cada imagen
    ax.axis('off')

    # Bucle que verifica que la imagen sea igual a 8
    while True:
        if data_y_test[j] == 8:
                        
            # Muestra la imagen en escala de grises
            ax.imshow(X_test_norm[j], cmap='Greys')
            j += 1
            break
        j += 1
plt.show()

#### Transformar los imagenes de 28x28 (2 dimensiones) en un array de una dimensión (28x28 = 784)
Esto se realiza porque las redes neuronales no soportan que se ingrese un array de dos dimensiones, solo soportan ingresar "N" features (un array)

In [ ]:
# proceso de flatten --> transformar las imagenes en un vector de 1 dimension
# shape devuelve filas y columnas
# Multiplica las filas por columnas (X_train_norm.shape[1] * X_train_norm.shape[2])
num_pixels = X_train_norm.shape[1] * X_train_norm.shape[2]

# X_train_norm.reshape(X_train_norm.shape[0], num_pixels) Ajusta el array a dos dimensiones
# Tipo de dato (astype('float32'))
X_train = X_train_norm.reshape(X_train_norm.shape[0], num_pixels).astype('float32')
X_test = X_test_norm.reshape(X_test_norm.shape[0], num_pixels).astype('float32')

In [ ]:
# ¿Cómo se ve ahora nuestra primera imagen?
fig = plt.figure()
ax = fig.add_subplot()

# Muestra la primer imagen de X_train con un ajuste .reshape(-1,1) y con escala de grises
ax.imshow(X_train[0].reshape(-1,1).T, cmap='gray')

# Escala de números en el eje de las x como potencia de base 10
ax.set_xscale("log")

# Agrega titulo
plt.title("Número: " + str(data_y_train[0]))

# Muetra la imagen
plt.show()

In [ ]:
# Devuelve información de la imagen.
print('Datos en observacion:', X_train.shape)

Son 60000 vectores, cada vector representa lo mismo que una fila de un dataset. Cada fila o vector tiene 784 columnas

In [ ]:
print('Dimensión de cada imagen faltten:', X_train[0].shape)

# Entrenar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline4.png" width="1000" align="middle">

Los datos ya estan dividios en train y test

In [ ]:
# Transformar la salida a oneHotEncoding con to_categorical
y_train = to_categorical(data_y_train)
y_test = to_categorical(data_y_test)

# Muestra los 10 primeros to_categorical
y_train[:10]

In [ ]:
# input shape (almacena la cantidad de pixeles de las imagen)
in_shape = X_train.shape[1]
in_shape

In [ ]:
# output shape, almacena la cantidad de número identificados en el dataset.
out_shape = y_train.shape[1]
out_shape

In [ ]:
# Se importa Dense de la librería tensorflow.keras.layers
from keras.layers import Dense, Dropout

# Se crea el objeto model a partir de la clase Sequential()
model = Sequential()

# Se crea la capa de entrada y las capas ocultas de la red, que tendrá:
# --> tantas entradas (input_shape) como columnas (in_shape), se especifica en la primera capa
# --> tantas neuronas como deseemos (units)
# --> utilizamos "relu" como capa de activación
model.add(Dense(units=64, activation='relu', input_shape=(in_shape,)))

# Se agregan dos capas más con las mismas neuronas y función de activación units=64, activation='relu'
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=64, activation='relu'))

# Se crea la capa de salida, que tendrá tantas neuronas como salidas posibles
# Se implementa 'softmax' ya que la salida es multiple
model.add(Dense(units=out_shape, activation='softmax'))

# Configuración del modelo para el entrenamiento, implementando el método compile a partir del modelo creado.
# Se necesita indicar los parámetros:
# optimizer, nombre del optimizador (es el algoritmo que se encarga del descenso de gradiente estocástico)
# Fuente: https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam
# loss, se llama función de pérdida, representa las categorías conocidas de las predicción. Al ser 'categorical_crossentropy' 
#la predicción tiene una salida con varias opciones.
# metrics, se define la métrica que evaluará el modelo durante el entrenamiento y las pruebas.
model.compile(optimizer="Adam",
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Resumen de la estructura de la red profunda.
model.summary()

In [ ]:
# Se entrena el modelo con el método fit
# Necesita definir los valores para X_train, y_train sumado a la cantidad de épocas que seria la iteraciones de entrenamiento.
# dirigido a validación (validation_split=0.2)
# batch_size, tamaño del lote a entrenar.
history = model.fit(X_train, y_train, validation_split=0.2 , epochs=10, batch_size=128)

In [ ]:
# Variable epoch_count, que almacena en una lista la cantidad de épocas de train
# history, es la variable que almacena las predicciones del modelo
# y de ella, se puede acceder a información como su historial (history) del accuracy
epoch_count = range(1, len(history.history['accuracy']) + 1)

# De Seaborn (sns) se accede al gráfico de línea para representar;
# Por un lado, el 'accuracy',
# Por el otro, la validación (val_accuracy)
sns.lineplot(x=epoch_count,  y=history.history['accuracy'], label='train')
sns.lineplot(x=epoch_count,  y=history.history['val_accuracy'], label='valid')
plt.show()

Se puede observar que el modelo con más capas ocultas tienen menos parámetros para entrar porque la capa expuesta a la entrada tiene menos neuronas.
Pero, al aumentar la complejidad de la red con más capas profundas el sistema produce overfitting, deja de aprender

In [ ]:
# Para arreglar esto se utiliza regularizacion, que es el proceso por el cual
# el sistema tiene la capacidad de "apagar" neuronas de las capas y buscar
# el modelo que mejor funcione.

# Se crea el objeto model a partir de la clase Sequential()
model = Sequential()

# Se seleccionó que se apagen el 20% de la neurona de la capa
# A este número se lo obtuvo realizando diferentes pruebas, en general se
# ensaya con --> 0.2, 0.5 y 0.8
dropout_rate = 0.2

# Red neuronal que tiene 4 capas densas y dos de dropout
model.add(Dense(units=64, activation='relu', input_shape=(in_shape,)))

model.add(Dense(units=64, activation='relu'))
model.add(Dropout(rate=dropout_rate))

model.add(Dense(units=64, activation='relu'))
model.add(Dropout(rate=dropout_rate))

model.add(Dense(units=out_shape, activation='softmax'))

model.compile(optimizer="Adam",
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
# Se entrena el modelo con el método fit
# Necesita definir los valores para X_train, y_train sumado a la cantidad de épocas que seria la iteraciones de entrenamiento.
# dirigido a validación (validation_split=0.2)
# batch_size, tamaño del lote a entrenar.
history = model.fit(X_train, y_train, validation_split=0.2 , epochs=10, batch_size=128)

In [ ]:
# Variable epoch_count, que almacena en una lista la cantidad de épocas de train
# history, es la variable que almacena las predicciones del modelo
# y de ella, se puede acceder a información como su historial (history) del accuracy
epoch_count = range(1, len(history.history['accuracy']) + 1)

# De Seaborn (sns) se accede al gráfico de línea para representar;
# Por un lado, el 'accuracy',
# Por el otro, la validación (val_accuracy)
sns.lineplot(x=epoch_count,  y=history.history['accuracy'], label='train')
sns.lineplot(x=epoch_count,  y=history.history['val_accuracy'], label='valid')
plt.show()

In [ ]:
# Al apagar neuronas de las capas se puede seguir aumentando la complejidad
# para ensayar
model = Sequential()

# Se seleccionó que se apagen el 20% de la neurona de la capa
# A este número se lo obtuvo realizando diferentes pruebas, en general se
# ensaya con --> 0.2, 0.5 y 0.8
dropout_rate = 0.2

# Red neuronal que tiene 7 capas densas y 5 de dropout
model.add(Dense(units=64, activation='relu', input_shape=(in_shape,)))
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(rate=dropout_rate))
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(rate=dropout_rate))
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(rate=dropout_rate))
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(rate=dropout_rate))
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(rate=dropout_rate))
model.add(Dense(units=out_shape, activation='softmax'))

model.compile(optimizer="Adam",
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
# Se entrena el modelo con el método fit
# Necesita definir los valores para X_train, y_train sumado a la cantidad de épocas que seria la iteraciones de entrenamiento.
# dirigido a validación (validation_split=0.2)
# batch_size, tamaño del lote a entrenar.
history = model.fit(X_train, y_train, validation_split=0.2 , epochs=10, batch_size=128)

In [ ]:
# Variable epoch_count, que almacena en una lista la cantidad de épocas de train
# history, es la variable que almacena las predicciones del modelo
# y de ella, se puede acceder a información como su historial (history) del accuracy
epoch_count = range(1, len(history.history['accuracy']) + 1)

# De Seaborn (sns) se accede al gráfico de línea para representar;
# Por un lado, el 'accuracy',
# Por el otro, la validación (val_accuracy)
sns.lineplot(x=epoch_count,  y=history.history['accuracy'], label='train')
sns.lineplot(x=epoch_count,  y=history.history['val_accuracy'], label='valid')
plt.show()

In [ ]:
# Variable y_hat_prob que almacena las probabilidades de las predicciones
# con los datos de evaluación
y_hat_prob = model.predict(X_test)
y_hat_prob[:3]

In [ ]:
# Muestra las clasificación de cada imagen, de acuerdo a la probabilidad más alta.
y_hat = np.argmax(y_hat_prob,axis=1)
y_hat[:3]

# Validar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline5.png" width="1000" align="middle">

In [ ]:
# Calcular la exactitud (accuracy)
scores = model.evaluate(X_test, y_test)
scores[1]

In [ ]:
# Se utiliza la matriz de confusión para evaluar la precisión de una clasificación.
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Necesita dos variables que contengan los valores a comparar
cm = confusion_matrix(y_test.argmax(axis=1), y_hat)

# Código para realizar la representación gráfica con los resultados
# Se crea la varible cmd, que almacena visualization de la Confusion Matrix 
# Necesita la variable cm que contiene los resultados de la comparación entre los valores reales y predicción
# display_labels, se especifica las etiquetas de las categorias que se evalúan.
cmd = ConfusionMatrixDisplay(cm, display_labels=list(range(10)))

# Con cmd.plot se especifica el mapa de colores reconocido por matplotlib.
cmd.plot(cmap=plt.cm.Blues)

# Mostrar la figura
plt.show()

# Utilizar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline6.png" width="1000" align="middle">

Se utiliza el ranking de los peores 10 números clasificados con una ANN para evaluar contra este nuevo modelo de red neuronal

In [ ]:
# Ubicación de los peores ochos por indice
ranking_10 = [8183, 6765, 8522, 1325, 582, 9280, 5749, 3567, 3206, 9744]

In [ ]:
# Obtener los vectores para evaluar
# Ubica el vector de cada imagen por su ubicación
X_test_peores = X_test[ranking_10]

# Ubica el número que corresponde a cada imagen por su ubicación
y_test_peores = y_test[ranking_10]

In [ ]:
# Calcular la exactitud
score = model.evaluate(X_test_peores, y_test_peores)
score[1]

In [ ]:
# ¿Qué es lo que el sistema ve?

# Predicción
y_hat_prob_peores = model.predict(X_test_peores)

# Ubica de acuerdo a la probabilidad más alta cuál es el número que le corresponde.
y_hat_peores = np.argmax(y_hat_prob_peores,axis=1)
y_hat_peores

In [ ]:
# Espacio para dibujar
fig = plt.figure(figsize=(16,9))

# Contador
j = 0

for i in ranking_10:

    # ax, gráfico que mostrará las imágenes en 1 filas y 10 columnas
    # En cada iteración va ubicando la imagen en la siguiente posición (i+1)
    ax = fig.add_subplot(1, 10, j+1)

    # .axis('off') elimina el recuadro de cada imagen
    ax.axis('off')

    # Muestra la imagen en escala de grises
    ax.imshow(X_test_norm[i], cmap='Greys')
    j += 1

plt.show()

### Prueba con imagen externa (Dibujar un número en paint con fondo negro y número en blanco)

In [ ]:
# Leer la imagen 
img1 = mpimg.imread('/content/cuatro.jpg')

# Mostrar la imagen
plt.imshow(img1)
plt.show()

In [ ]:
# Ver las dimensiones de la imagen cargada
img1.shape

In [ ]:
# Función que extrae los canales de color
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

In [ ]:
# Se invoca la función y se le pasa la imagen
gray = rgb2gray(img1)  
gray.shape

In [ ]:
# Debemos ahora quitar los canales de color para que sea en escala de grises
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

# Se invoca la función
gray = rgb2gray(img1)    

# cmap='gray', variación de colores en grises, indicando los valores máximo y mínimo de la escala de colores.
plt.imshow(gray, cmap='gray', vmin=0, vmax=255)
plt.show()

In [ ]:
#Modificando las dimensiones de la imagen para que sea 28x28
copi_img = gray[:,:28]
copi_img.shape

In [ ]:
# Preprocesar la imagen
# Normalizar
img_norm = copi_img / 255.0

# Calcular el número de pixeles de la imagen.
num_pixel = img_norm.shape[0] * img_norm.shape[1]
num_pixel

In [ ]:
# Ajustar la imagen en una fila.
img_prueba = img_norm.reshape(1,num_pixel).astype('float32')
img_prueba.shape

In [ ]:
# ¿Qué es lo que el sistema ve?
# Predicción
prediccion = model.predict([img_prueba])
prediccion[0]

In [ ]:
# Muestra las clasificación de cada imagen, de acuerdo a la probabilidad más alta.
y_prediccion = np.argmax(prediccion,axis=1)
y_prediccion[0]

# Conclusión
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline7.png" width="1000" align="middle">

Al utilizar deep larning, redes profundas, logramos obtener un mejor resultado. __PERO!__ Al utilizar más capas fue necesario el proceso de regularización con dropout para poder evitar que un modelo más complejo produza overfiting. <br>
Este modelo sigue siendo incapaz de manejar imagenes a color debido a la necesidad del proceso de flatten.